In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

In [64]:
data = pd.read_csv('dataset/train.csv')

In [52]:
test = pd.read_csv('dataset/test.csv')

In [78]:
test

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,...,Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII)
0,1007012,KK98 77,0.456086,51.562543,12236.576447,0.593325,10.414164,10.699072,0.547114,0.556267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1007012,Reticulum III,0.529835,57.228262,3431.883825,0.675407,7.239485,5.311122,0.497688,0.409969,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1008016,Reticulum III,0.560976,59.379539,27562.914252,0.594624,11.774890,5.937797,0.544744,0.486167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1007012,Segue 1,0.565910,59.952390,20352.232905,0.837700,11.613621,10.067882,0.691641,0.523441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1013042,Virgo I,0.588274,55.428320,23959.704016,0.520579,10.392416,6.374637,0.530676,0.580418,...,7.357729,0.583373,0.600445,0.856158,206.674424,224.104054,NaN,7.687626,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,1016064,ESO 383-087 (ISG 39),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886,1016064,NGC 247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
887,1016064,ESO 325-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
888,1016064,ESO 540-032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
test.describe()

,galactic year,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",...,Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII)
count,8.900000e+02,885.000000,885.000000,885.000000,885.000000,885.000000,882.000000,882.000000,882.000000,880.000000,...,606.000000,560.000000,560.000000,560.000000,549.000000,549.000000,519.000000,482.000000,536.000000,529.000000
mean,1.011498e+06,0.923881,79.938370,33655.513114,0.856869,16.037675,11.229365,0.845626,0.801581,147.296372,...,8.936240,0.848007,0.862777,1.015743,128.712080,128.635268,20.640391,6.036133,23.274262,0.563059
std,2.881505e+03,0.134856,9.051945,18940.572797,0.186299,3.170799,3.333436,0.164942,0.192220,57.532201,...,2.610288,0.185827,0.151898,0.100165,48.871212,48.684267,15.365777,4.157453,13.787944,0.203167
min,1.007012e+06,0.456086,51.562543,734.072777,0.357646,6.902194,3.224037,0.452401,0.268006,5.235352,...,2.820374,0.180104,0.491085,0.319004,6.740082,18.526901,-62.918138,-0.455652,-74.254897,0.139063
25%,1.009020e+06,0.834118,73.863054,21276.527313,0.710236,13.711596,8.634776,0.711922,0.648948,101.506337,...,7.037902,0.711255,0.739310,0.970260,88.348928,90.098284,14.394432,4.075216,16.943013,0.380811
50%,1.011030e+06,0.944683,81.471317,28264.476530,0.868819,16.269818,11.274397,0.863623,0.817313,147.972770,...,8.685332,0.879984,0.888459,1.039880,129.450600,130.491213,21.599869,5.185122,23.714018,0.569413
75%,1.014049e+06,1.022712,86.426984,41130.714108,0.997650,18.272034,14.067355,0.977608,0.953022,195.585489,...,10.769263,0.995167,0.976561,1.079805,169.529106,168.104504,28.811899,6.601704,30.746076,0.733748
max,1.016064e+06,1.250508,100.427653,134989.054424,1.307948,24.802112,18.426796,1.198469,1.188590,263.029425,...,18.683942,1.220746,1.185086,1.171331,234.055255,237.382202,62.944910,31.487193,66.751764,1.088255


In [56]:
data.iloc[:, -1].describe()

count    3865.000000
mean        0.082773
std         0.063415
min         0.013036
25%         0.047889
50%         0.057820
75%         0.087389
max         0.683813
Name: y, dtype: float64

In [81]:
data[data['galactic year'] == 1015056]

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,...,"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
3778,1015056,Andromeda XII,1.122844,89.594199,29407.532402,0.791718,14.824049,9.086485,0.850167,0.733780,...,0.926955,0.883561,1.004556,120.234697,163.729690,NaN,NaN,-0.919483,0.569571,0.061414
3779,1015056,Ursa Major I Dwarf (UMa I dSph),1.123090,79.287116,34614.041801,0.980279,18.579410,9.525747,0.901291,0.926239,...,0.868072,0.934047,1.056369,123.473361,120.693643,15.525371,1.934369,26.086254,0.535346,0.040206
3780,1015056,Andromeda IX,1.147190,92.698990,44033.925422,1.025085,19.263960,15.248176,1.037091,0.882128,...,0.985079,1.104151,0.993973,62.538556,87.147696,20.443243,2.538673,14.804013,0.288618,0.285940
3781,1015056,Andromeda XIX[60],1.104299,87.937012,52941.889557,1.139313,19.927652,13.234882,1.086366,0.982908,...,1.057919,0.986739,0.993855,83.846372,52.265903,NaN,NaN,-26.332212,0.338689,0.194908
3782,1015056,Ursa Major II Dwarf,1.126053,91.477794,65709.423048,1.074623,26.658594,15.583388,1.066657,1.080778,...,1.170121,1.047470,1.035972,63.016682,82.543885,20.995878,5.814485,27.415489,0.320044,0.204002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3860,1015056,Columba I,1.029704,82.832063,34310.471408,0.855094,18.578586,10.557143,0.906573,0.862826,...,0.943410,0.902237,1.060532,124.564121,143.907576,26.438719,3.023709,29.294865,0.580785,0.042324
3861,1015056,"Leo II Dwarf (Leo B, DDO 93)",0.937869,75.877098,36899.067719,0.929494,16.153857,9.151665,0.865822,0.747577,...,0.915225,0.798083,1.055118,163.664516,184.291155,20.637654,4.470596,31.085400,0.517558,0.036725
3862,1015056,Canes Venatici I Dwarf,1.036144,93.540275,37002.977875,1.085245,21.066473,16.661344,0.983835,1.100779,...,1.097208,1.044890,1.114754,66.498714,112.887035,28.154859,5.193997,32.145570,0.363862,0.166271
3863,1015056,KKs 3,0.939034,78.274427,28180.459770,0.687655,9.388911,8.908748,0.735694,0.602703,...,0.596164,0.754729,0.825864,182.249079,175.408953,38.963157,2.854140,27.227179,0.711878,0.024187


In [80]:
data[data['galaxy'] == 'Reticulum III']

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,...,"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
172,990025,Reticulum III,0.522122,56.730263,24900.347144,0.533632,7.080165,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.37007,6.394217,NaN,NaN,0.050080
330,991020,Reticulum III,0.543908,59.543367,19692.923240,0.500455,6.904359,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049181
528,992016,Reticulum III,0.603091,52.578569,9106.962730,0.569982,7.155640,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049087
610,993012,Reticulum III,0.480042,54.145697,26787.530987,0.537528,6.239010,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.048732
869,994009,Reticulum III,0.487961,59.387290,17051.697422,0.640835,6.481115,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.048368
1007,995006,Reticulum III,0.569395,56.402142,17575.780473,0.489286,6.547220,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.584252,NaN,NaN,0.048065
1173,996004,Reticulum III,0.611309,52.049187,6478.718803,0.587405,6.081191,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046925
1304,997002,Reticulum III,0.658729,54.915558,20244.581052,0.500271,6.308482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046594
1606,998001,Reticulum III,0.540846,59.247423,25723.326763,0.550751,9.565715,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046181
1700,999000,Reticulum III,0.509840,57.068819,12407.825863,0.565098,8.392391,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045822


In [93]:
data.groupby('galaxy')['y'].max().sort_values()

galaxy
NGC 5253        0.045115
Pegasus III     0.045860
Antlia B        0.047487
IC 3104         0.047695
Leo V Dwarf     0.048458
                  ...   
Leo T Dwarf     0.399245
ESO 294-010     0.473119
NGC 3741        0.477482
Tucana Dwarf    0.525346
Cetus Dwarf     0.683813
Name: y, Length: 181, dtype: float64

In [98]:
data['galaxy'].unique()

array(['Large Magellanic Cloud (LMC)', 'Camelopardalis B', 'Virgo I',
       'UGC 8651 (DDO 181)', 'Tucana Dwarf', 'KKh 060', 'Grus II',
       'UGCA 292', 'Aquarius II', 'Andromeda XI', 'Antlia Dwarf',
       'IC 1613 (UGC 668)', 'Leo P',
       'Cassiopeia Dwarf (Cas dSph, Andromeda VII)', 'Draco II',
       'Sagittarius II', 'Carina III', 'UGCA 133 (DDO 44)',
       'Lacerta I (Andromeda XXXI)', 'Boötes IV', 'Maffei 2',
       'KUG 1210+301B (KK98 127)', 'Andromeda V',
       'Sagittarius Dwarf Sphr SagDEG', 'UGCA 276 (DDO 113)',
       'Ursa Minor Dwarf', 'UGC 9128 (DDO 187)', 'Andromeda XX',
       'Segue 2', 'Fornax Dwarf (E356-G04)', 'Columba I', 'KK98 77',
       'Holmberg II (DDO 50, UGC 4305)', 'ESO 540-030 (KDG 2)', 'NGC 404',
       'UGC 8833', 'Perseus I (Andromeda XXXIII)', 'Horologium II',
       'Andromeda XXVI', 'Grus I', 'Andromeda IX', 'Andromeda XXII[57]',
       'ESO 274-01[70]', 'Leo II Dwarf (Leo B, DDO 93)', 'Reticulum II',
       'Andromeda II', 'Indus II', 'MB

In [99]:
test['galaxy'].unique()

array(['KK98 77', 'Reticulum III', 'Segue 1', 'Virgo I',
       'Wolf-Lundmark-Melotte (WLM, DDO 221)', 'KKR 25', 'Antlia B',
       'Hydra II', 'UGC 8651 (DDO 181)', 'NGC 300', 'Leo V Dwarf',
       'KKh 060', 'Cetus III', 'Crater II', 'Andromeda XVII',
       'Andromeda XXIX', 'Draco Dwarf (DDO 208)', 'Andromeda VIII',
       'KKH 12', 'Pegasus III', 'Cassiopeia II (Andromeda XXX)',
       'IC 3104', 'UGCA 133 (DDO 44)', 'UGCA 92',
       "Barnard's Galaxy (NGC 6822)", 'KKs 3',
       'Leo I Dwarf (DDO 74, UGC 5470)', 'Sextans Dwarf Sph',
       'Horologium II', 'Andromeda X', 'UGC 9240 (DDO 190)',
       'Andromeda XI', 'NGC 1569 (UGC 3056)', 'Carina Dwarf (E206-G220)',
       'IC 1613 (UGC 668)', 'Tucana IV',
       'Aquarius Dwarf Galaxy (DDO 210)', 'UGC 4483',
       'Andromeda Galaxy (M31)', 'Andromeda I', 'Draco II',
       'Large Magellanic Cloud (LMC)', 'Lacerta I (Andromeda XXXI)',
       'ESO 383-087 (ISG 39)', 'ESO 274-01[70]', 'ESO 540-030 (KDG 2)',
       'Pisces V (Andr

In [50]:
unique_years = data['galactic year'].unique()
print (unique_years)
print (unique_years.shape)

[ 990025  991020  992016  993012  994009  995006  996004  997002  998001
  999000 1000000 1001000 1002001 1003002 1004004 1005006 1006009 1007012
 1008016 1009020 1010025 1011030 1012036 1013042 1014049 1015056]
(26,)


In [77]:
print(np.sort(test['galactic year'].unique()))
print(test['galactic year'].unique().shape)

[1007012 1008016 1009020 1010025 1011030 1012036 1013042 1014049 1015056
 1016064]
(10,)


In [58]:
(unique_years - 990025)/1001

array([ 0.        ,  0.99400599,  1.98901099,  2.98401598,  3.98001998,
        4.97602398,  5.97302697,  6.97002997,  7.96803197,  8.96603397,
        9.96503497, 10.96403596, 11.96403596, 12.96403596, 13.96503497,
       14.96603397, 15.96803197, 16.97002997, 17.97302697, 18.97602398,
       19.98001998, 20.98401598, 21.98901099, 22.99400599, 24.        ,
       25.00599401])